In [1]:
# import necessary packages
import sys, os
import torch 
import numpy as np
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer, setup_chat_format
from transformers import (pipeline,
                          AutoTokenizer,
                          AutoModelForCausalLM,
                          DataCollatorForLanguageModeling,
                          DataCollatorWithPadding,
                          get_scheduler)
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from IPython.display import clear_output

sys.path.append('../')

# custom imports
from utils.GetLowestGPU import GetLowestGPU

device = GetLowestGPU()

Device set to cuda:0


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Instantiate Model and Dataset

In [3]:
# options
model_path = "meta-llama/Meta-Llama-3-8B"
dataset_path = "allenai/peS2o"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

# load tokenizer and model
pipeline = pipeline('text-generation', 
                    model=model_path,
                    model_kwargs={'torch_dtype': torch.bfloat16},
                    device_map = 'auto'
                    )

pipeline.model = get_peft_model(pipeline.model, peft_config)
pipeline.tokenizer.pad_token = pipeline.tokenizer.eos_token
pipeline.tokenizer.pad_token_id = pipeline.tokenizer.eos_token_id
pipeline.model.generation_config.pad_token_id = pipeline.tokenizer.eos_token_id

pipeline.model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


In [4]:
# load dataset
raw_dataset = load_dataset(dataset_path, "v2", streaming=True, trust_remote_code=True)

# check format of data
raw_dataset

IterableDatasetDict({
    train: IterableDataset({
        features: ['added', 'created', 'id', 'source', 'text', 'version'],
        n_shards: 20
    })
    validation: IterableDataset({
        features: ['added', 'created', 'id', 'source', 'text', 'version'],
        n_shards: 2
    })
})

# Preprocessing

In [5]:
# define functions
def preprocess_data(examples):
    tokenized_data = pipeline.tokenizer(text=examples['text'],
                               padding='max_length', 
                               truncation=True, 
                               max_length=100)
    
    labels = tokenized_data['input_ids'].copy()
    
    # for i in range(len(labels)):
    #     if labels[i][-1] != pipeline.tokenizer.pad_token_id:
    #         labels[i] = labels[i][1:] + [pipeline.tokenizer.pad_token_id]
    #     else:
    #         labels[i] = labels[i][1:] + [pipeline.tokenizer.pad_token_id]

    # labels = [[pipeline.tokenizer.pad_token_id if x == pipeline.tokenizer.pad_token_id else x for x in y] for y in labels]
    # tokenized_data['labels'] = labels
    
    return tokenized_data

In [6]:
# add special tokens to tokenizer
pipeline.tokenizer.pad_token = pipeline.tokenizer.eos_token
pipeline.model.resize_token_embeddings(len(pipeline.tokenizer))

tokenized_dataset = raw_dataset.map(preprocess_data,
                                    batched=True,
                                    remove_columns=raw_dataset['train'].column_names,)
tokenized_dataset.with_format("torch")

# check tokenized dataset output
tokenized_dataset["train"]

IterableDataset({
    features: Unknown,
    n_shards: 20
})

# Create Dataloaders

In [7]:
# instantiate data collator
data_collator = DataCollatorWithPadding(tokenizer=pipeline.tokenizer)

train_dataloader = DataLoader(tokenized_dataset['train'],
                              batch_size=8, 
                              collate_fn=data_collator,
                              num_workers=20)

val_dataloader = DataLoader(tokenized_dataset['validation'],
                            batch_size=8,
                            collate_fn=data_collator,
                            num_workers=2)

In [8]:
# inspect sample batch
batch = next(iter(train_dataloader))
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 100]), 'attention_mask': torch.Size([8, 100])}

In [9]:
outputs = pipeline.model(batch['input_ids'].to(device), labels=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
print(outputs.loss, outputs.logits.shape)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU  has a total capacity of 39.50 GiB of which 3.81 MiB is free. Process 2933268 has 37.91 GiB memory in use. Including non-PyTorch memory, this process has 1.57 GiB memory in use. Of the allocated memory 1.03 GiB is allocated by PyTorch, and 49.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Training

In [10]:
# run a test prediction
text = ["Network biology is the study of"]

terminators = [
    pipeline.tokenizer.eos_token_id
]

outputs = pipeline(
    text,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0][0]['generated_text'])

Network biology is the study of biological systems in which the focus is on the interactions between the components of the system rather than the components themselves. The systems are represented as networks where the nodes are the components of the system and the edges are the interactions between them. In this thesis, we study network biology from a computational perspective. We develop new methods for analyzing biological networks and apply them to real-world problems. We first present a new method for detecting communities in biological networks. Communities are groups of nodes that are more densely connected to each


In [11]:
# pipeline.model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
#     pipeline.model, optimizer, train_dataloader, lr_scheduler)

In [12]:
# options
num_batches = 1_000
num_epochs = 1
best_val_loss = np.inf
checkpoint_path = '../checkpoints/checkpoint_{0}.pt'
log_path = '../logs/log.csv'

# init optimizer
optimizer = AdamW(pipeline.model.parameters(), lr=1e-5)

# init scheduler
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=1000,
    num_training_steps=num_epochs * num_batches,
)

with open(log_path, 'w') as f: 
    f.write(f'epoch,iter_num,train_loss,val_loss\n')

# loop
for epoch in range(num_epochs):

    clear_output(wait=True)

    print("=====================")
    print(f"Epoch {epoch + 1}")
    print("=====================")

    # initialize train loss, val loss
    running_train_loss = 0.0
    running_val_loss = 0.0

    # loop through train data
    print("Training...")
    i = 0
    with tqdm(total=num_batches) as pbar:
        for train_batch, val_batch in zip(train_dataloader, val_dataloader):
            
            ## training
            # set model to train mode
            pipeline.model.train()

            # grab batch and map to device
            train_batch = {k: v.to(device) for k, v in train_batch.items()}

            # forward pass
            outputs = pipeline.model(train_batch['input_ids'], 
                                     labels=train_batch['input_ids'],
                                     attention_mask=train_batch['attention_mask'])
            train_loss = outputs.loss

            running_train_loss += train_loss.item()

            # backward pass
            train_loss.backward()
            # accelerator.backward(loss)

            # clip gradients
            torch.nn.utils.clip_grad_norm_(pipeline.model.parameters(), 1.0)

            # update optimizer, scheduler
            optimizer.step()
            lr_scheduler.step()

            # zero gradients
            optimizer.zero_grad()
            
            ## validation
            # set model to eval mode
            pipeline.model.eval()
            # loop through val data
            val_batch = {k: v.to(device) for k, v in val_batch.items()}
            with torch.no_grad():
                outputs = pipeline.model(val_batch['input_ids'], 
                                         labels=val_batch['input_ids'],
                                         attention_mask=val_batch['attention_mask'])
                val_loss = outputs.loss
                running_val_loss += val_loss.item()
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
            
            print(f"Train Batch Loss: {train_loss:.4f} | Val Batch Loss: {val_loss:.4f} | Best Val. Loss: {best_val_loss:.4f}\r", end="")

            i += 1
            pbar.update(1)
            if i % 1000 == 0:

                # # save model checkpoint
                # checkpoint = {
                #     'model': pipeline.model.state_dict(),
                #     'optimizer': optimizer.state_dict(),
                #     'epoch': epoch,
                #     'iter_num': i,``
                #     'best_val_loss': best_val_loss,
                # }
                # torch.save(checkpoint, checkpoint_path.format(i))

                # print example output
                print(f"Batch {i} of {num_batches}; Printing Example Response...")
                print(pipeline(text,
                               max_new_tokens=256,
                               eos_token_id=terminators,
                               no_repeat_ngram_size=3,       
                               do_sample=True, 
                               top_k=100, 
                               top_p=0.9,
                               temperature=0.6)[0][0]['generated_text'])

            # write to log
            with open(log_path, 'a') as f: 
                f.write(f'{epoch},{i},{train_loss},{val_loss}\n')
            
            if i == num_batches:
                print(f"Reached {num_batches} batches; breaking...")
                break
    
    train_loss = running_train_loss / num_batches
    val_loss = running_val_loss / num_batches

    train_loss = running_train_loss / len(train_dataloader)
    print(f"Avg. Train Loss: {train_loss:.4f}, Avg. Val Loss: {val_loss:.4f}")
    # print("Evaluation metrics:", metric.compute())

print("Training Complete!")

Epoch 1
Training...


  0%|          | 0/1000 [00:00<?, ?it/s]

/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Batch 1000 of 1000; Printing Example Response...0 | Best Val. Loss: 1.7329
Network biology is the study of the biological network and its role in the understanding of complex biological systems. The use of network theory in the study and modeling of biological systems has grown significantly in the past decade. This book provides a comprehensive introduction to the network biology approach, including the basic concepts and methods, as well as the application of network biology in different biological systems, such as gene regulatory networks, metabolic networks, and signaling networks. The book also discusses the latest advances in network biology, such...

The purpose of this book is to introduce the basic knowledge and concepts of bioinformatics and systems biology to the readers. Bioinformatics is a new interdisciplinary field that is formed by the integration of biology, computer science, and information technology. Systems biology is a novel research field that focuses on the comp

TypeError: object of type 'IterableDataset' has no len()

In [13]:
# run a test prediction
outputs = pipeline(
    text,
    max_new_tokens=100,
    eos_token_id=terminators,
    no_repeat_ngram_size=3,       
    do_sample=True, 
    top_k=100, 
    top_p=0.9,
    temperature=0.6
)
print(outputs[0][0]['generated_text'])

Network biology is the study of biological systems at the level of networks. This article reviews the application of network biology to the study and understanding of the immune system. We first discuss the various types of biological networks, their properties and their roles in the immune response. We then review the main computational approaches used to analyze and understand the networks, with a focus on the use of the concept of network motifs. Finally, we discuss the current challenges and future directions of network immunology research.
